In MySQL, events are scheduled tasks that run at specific intervals or at a particular time, much like cron jobs in Unix-based systems. These events allow you to automate database tasks such as cleanup, updating records, or running periodic reports.

### Steps to Create Events in MySQL

1. **Enable the Event Scheduler**:
   Before you can create events, ensure that the event scheduler is enabled. You can check if it's enabled with this query:

   ```sql
   SHOW VARIABLES LIKE 'event_scheduler';
   ```

   If the event scheduler is off, you can enable it by running:

   ```sql
   SET GLOBAL event_scheduler = ON;
   ```

   Alternatively, you can configure this setting in the MySQL configuration file (`my.cnf` or `my.ini`) under the `[mysqld]` section:

   ```ini
   [mysqld]
   event_scheduler = ON
   ```

2. **Create an Event**:
   An event can be created using the `CREATE EVENT` statement. The basic syntax is as follows:

   ```sql
   CREATE EVENT event_name
   ON SCHEDULE schedule_expression
   DO
   event_body;
   ```

   - `event_name`: The name of the event.
   - `schedule_expression`: Defines when and how often the event will run.
   - `event_body`: The SQL statements that will be executed by the event.

   Here are two common types of events:

   - **One-Time Event**: This event runs once at a specific time.

     ```sql
     CREATE EVENT my_event
     ON SCHEDULE AT '2024-09-21 10:00:00'
     DO
     UPDATE my_table SET status = 'inactive' WHERE last_login < '2024-01-01';
     ```

   - **Recurring Event**: This event runs periodically at defined intervals.

     ```sql
     CREATE EVENT my_recurring_event
     ON SCHEDULE EVERY 1 DAY
     DO
     DELETE FROM my_table WHERE last_login < NOW() - INTERVAL 1 YEAR;
     ```

   - **Other Scheduling Options**:
     - `EVERY INTERVAL`: Specifies the interval, e.g., `EVERY 1 HOUR`, `EVERY 7 DAY`.
     - `STARTS`: Specifies when the event should start.
     - `ENDS`: Specifies when the event should stop running.

     Example of a recurring event with a defined start and end time:

     ```sql
     CREATE EVENT my_limited_event
     ON SCHEDULE EVERY 1 DAY
     STARTS '2024-09-21 00:00:00'
     ENDS '2024-12-31 23:59:59'
     DO
     UPDATE my_table SET status = 'archived' WHERE created_at < NOW() - INTERVAL 6 MONTH;
     ```

3. **Viewing Events**:
   You can view the created events using:

   ```sql
   SHOW EVENTS;
   ```

   Or you can see the event details for a specific database:

   ```sql
   SHOW EVENTS FROM my_database;
   ```

4. **Modifying an Event**:
   To modify an event, use the `ALTER EVENT` statement:

   ```sql
   ALTER EVENT event_name
   ON SCHEDULE EVERY 1 WEEK
   DO
   UPDATE my_table SET status = 'inactive';
   ```

5. **Dropping an Event**:
   To remove an event, use the `DROP EVENT` statement:

   ```sql
   DROP EVENT IF EXISTS my_event;
   ```

### Additional Options:

- **Disabling/Enabling an Event**:
   You can disable or enable an event without dropping it.

   ```sql
   ALTER EVENT event_name DISABLE;
   ALTER EVENT event_name ENABLE;
   ```

- **Event Privileges**:
   To create or drop events, the user must have the `EVENT` privilege. You can grant this privilege with:

   ```sql
   GRANT EVENT ON database_name.* TO 'user'@'host';
   ```

Using these steps, you can automate various database management tasks with scheduled events in MySQL.

In MySQL, the use of `BEGIN ... END` is required when you want to include multiple SQL statements within an event (or a stored procedure or trigger). However, if the event contains only a **single SQL statement**, you don't need to wrap it in a `BEGIN ... END` block. 

### When to Use `BEGIN ... END`

- **Single SQL Statement**:
  If your event contains only one SQL statement, `BEGIN ... END` is not required. Here’s an example of a single-statement event:

  ```sql
  CREATE EVENT my_event
  ON SCHEDULE EVERY 1 DAY
  DO
  UPDATE my_table SET status = 'inactive' WHERE last_login < NOW() - INTERVAL 1 YEAR;
  ```

  Since only one statement is present after `DO`, `BEGIN ... END` is not necessary.

- **Multiple SQL Statements**:
  If you want the event to execute multiple SQL statements, then you need to wrap them in a `BEGIN ... END` block to define the scope of the statements. Here's an example of an event with multiple statements:

  ```sql
  CREATE EVENT my_complex_event
  ON SCHEDULE EVERY 1 DAY
  DO
  BEGIN
    UPDATE my_table SET status = 'inactive' WHERE last_login < NOW() - INTERVAL 1 YEAR;
    DELETE FROM logs WHERE log_date < NOW() - INTERVAL 1 YEAR;
  END;
  ```

  In this case, `BEGIN ... END` is required to enclose the two statements and treat them as a block. Without it, the MySQL server would not know how to handle the second SQL statement, and an error would occur.

### Summary:
- **Single SQL statement**: No need for `BEGIN ... END`.
- **Multiple SQL statements**: You must use `BEGIN ... END` to group the statements together within the `DO` clause.

Using `BEGIN ... END` depends on the number of statements you're including in the event.